In [1]:
import importlib
import commai_spd_chl_keras_datagen 
importlib.reload(commai_spd_chl_keras_datagen)

import keras
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential,load_model
import tensorflow.keras.backend as K
from commai_spd_chl_keras_datagen import commai_speed_chl_gen
from tensorflow.keras.layers import Dense,Activation,Lambda,GRU,BatchNormalization,Activation,Flatten,Conv2D,MaxPooling2D,Reshape,Dropout

Using TensorFlow backend.


In [2]:
tf.config.list_physical_devices ()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.keras.backend.image_data_format()

'channels_last'

In [4]:
seq = Sequential()

#should I trust this guy?https://www.kdnuggets.com/2018/09/dropout-convolutional-networks.html

seq.add(Conv2D(filters=32,kernel_initializer='he_normal', kernel_size=(4,4),padding='same',input_shape=(480,640,3)))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=32, kernel_size=(4,4),kernel_initializer='he_normal',padding='same'))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=64, kernel_size=(4,4),kernel_initializer='he_normal',padding='same'))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=64, kernel_size=(4,4),kernel_initializer='he_normal',padding='same'))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Flatten())

seq.add(Dense(192,kernel_initializer='he_normal'))
seq.add(Activation("relu"))
seq.add(BatchNormalization())
seq.add(Dropout(.3)) # throw away 30%

seq.add(Dense(50,kernel_initializer='he_normal'))
seq.add(Activation("relu"))
seq.add(Dropout(.2))

seq.add(Dense(1,kernel_initializer='he_normal'))

seq.compile(loss='mse', optimizer='adam')
seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 480, 640, 32)      1568      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 120, 160, 32)      0         
_________________________________________________________________
activation (Activation)      (None, 120, 160, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 120, 160, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 120, 160, 32)      16416     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 40, 32)        0         
_________________________________________________________________
activation_1 (Activation)    (None, 30, 40, 32)        0

In [12]:
commai_speed_chl_gen(v="data_new/train.mp4",db_name="/mnt/imgs",txt="data_new/train.txt").build_video_folder()

  0%|          | 15/53831 [00:00<06:14, 143.72it/s]

(480, 640)
(32, 3, 480, 640)
(32, 3, 1)
making dir:, /mnt/imgs
building video Database


100%|██████████| 53831/53831 [11:50<00:00, 75.79it/s] 


In [5]:
batch_size=45
n_e=120
# total new frames:53830
n_frames_train=45600
n_frames_test=4000
n_frames_valid=4000

train_idx,test_idx,valid_idx=commai_speed_chl_gen(n_frames_valid=n_frames_valid,frames_per_sample=2,mode="optical_flow_dense",
                    n_frames_test=n_frames_test,n_frames_train=n_frames_train,batch_size=batch_size,
                                                  v="data_new/train.mp4",txt="data_new/train.txt",
                                                  db_name="/mnt/imgs").get_indexes()

load=np.load(open("idxes_45bs_opflowdense_KITTI.np","rb"))
train_idx=load["train_idx"]
test_idx=load["test_idx"]
valid_idx=load["valid_idx"]

train_gen=commai_speed_chl_gen(v="data_new/train.mp4",txt="data_new/train.txt",db_name="/mnt/imgs",
                frame_idx=train_idx,batch_size=batch_size,mode="optical_flow_dense",frames_per_sample=2)
test_gen=commai_speed_chl_gen(v="data_new/train.mp4",txt="data_new/train.txt",db_name="/mnt/imgs",
                frame_idx=test_idx,batch_size=batch_size,mode="optical_flow_dense",frames_per_sample=2)
valid_gen=commai_speed_chl_gen(v="data_new/train.mp4",txt="data_new/train.txt",db_name="/mnt/imgs",
                frame_idx=valid_idx,batch_size=batch_size,mode="optical_flow_dense",frames_per_sample=2)

#np.savez_compressed(open("idxes_45bs_opflowdense_KITTI.np","wb+"),train_idx=train_idx,test_idx=test_idx,valid_idx=valid_idx)



53760
(480, 640, 3)
(45, 480, 640, 3)
(45, 1)
train_samples: 22800 
test_samples:2000 
valid_samples: 2000
53760
(480, 640, 3)
(45, 480, 640, 3)
(45, 1)
53760
(480, 640, 3)
(45, 480, 640, 3)
(45, 1)
53760
(480, 640, 3)
(45, 480, 640, 3)
(45, 1)


In [6]:
a=[]
import time
t=time.time()
for i in test_gen:
    a.append(i)
    break
print(time.time()-t) # fuck me 

4.71239161491394


In [7]:
ie=0
filepath="STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-{epoch:02d}-{val_loss:.2f}-{loss:.2f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False)

tb=tf.keras.callbacks.TensorBoard(
    log_dir='STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI', histogram_freq=1, write_graph=True, write_images=False,
    update_freq=10, profile_batch=0, embeddings_freq=0
)

cbs=[checkpoint,tb]
hst=seq.fit_generator(train_gen, epochs=n_e,workers=25,max_queue_size=100,
        use_multiprocessing=True,validation_data=test_gen,callbacks=cbs,verbose=1,shuffle=True, initial_epoch=ie)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/120
506/506 [==============================] - ETA: 0s - loss: 134.8327WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00001: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-01-66.44-134.83.hdf5
506/506 [==============================] - 687s 1s/step - loss: 134.8327 - val_loss: 66.4380
Epoch 2/120
506/506 [==============================] - ETA: 0s - loss: 95.2243WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00002: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-02-45.06-95.22.hdf5
506/506 [==============================] - 686s 1s/step - loss: 95.2243 - val_loss: 45.0551
Epoch 3/1


Epoch 00008: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-08-24.57-36.51.hdf5
506/506 [==============================] - 683s 1s/step - loss: 36.5060 - val_loss: 24.5745
Epoch 9/120
506/506 [==============================] - ETA: 0s - loss: 32.2420WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00009: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-09-25.74-32.24.hdf5
506/506 [==============================] - 694s 1s/step - loss: 32.2420 - val_loss: 25.7409
Epoch 10/120
506/506 [==============================] - ETA: 0s - loss: 29.6216WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00010: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dens

506/506 [==============================] - 704s 1s/step - loss: 20.1024 - val_loss: 18.7028
Epoch 17/120
506/506 [==============================] - ETA: 0s - loss: 19.0820WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00017: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-17-17.67-19.08.hdf5
506/506 [==============================] - 701s 1s/step - loss: 19.0820 - val_loss: 17.6653
Epoch 18/120
506/506 [==============================] - ETA: 0s - loss: 18.4723WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00018: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-18-18.86-18.47.hdf5
506/506 [==============================] - 702s 1s/step - loss: 18.4723 - val_loss: 18

Epoch 25/120
506/506 [==============================] - ETA: 0s - loss: 14.6072WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00025: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-25-16.80-14.61.hdf5
506/506 [==============================] - 690s 1s/step - loss: 14.6072 - val_loss: 16.8013
Epoch 26/120
506/506 [==============================] - ETA: 0s - loss: 13.3998WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00026: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-26-17.62-13.40.hdf5
506/506 [==============================] - 698s 1s/step - loss: 13.3998 - val_loss: 17.6237
Epoch 27/120
506/506 [==============================] - ETA: 0s - loss: 13.1178WARNING

506/506 [==============================] - ETA: 0s - loss: 11.5848WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00033: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-33-14.73-11.58.hdf5
506/506 [==============================] - 703s 1s/step - loss: 11.5848 - val_loss: 14.7283
Epoch 34/120
506/506 [==============================] - ETA: 0s - loss: 11.4964WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00034: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-34-18.41-11.50.hdf5
506/506 [==============================] - 685s 1s/step - loss: 11.4964 - val_loss: 18.4065
Epoch 35/120
506/506 [==============================] - ETA: 0s - loss: 11.2171WARNING:tensorflow:m

506/506 [==============================] - ETA: 0s - loss: 8.8105WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00049: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-49-14.78-8.81.hdf5
506/506 [==============================] - 705s 1s/step - loss: 8.8105 - val_loss: 14.7777
Epoch 50/120
506/506 [==============================] - ETA: 0s - loss: 8.7673WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00050: saving model to STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-50-15.37-8.77.hdf5
506/506 [==============================] - 695s 1s/step - loss: 8.7673 - val_loss: 15.3711
Epoch 51/120
506/506 [==============================] - ETA: 0s - loss: 8.7998WARNING:tensorflow:multipro

Process Keras_worker_ForkPoolWorker-5419:
Process Keras_worker_ForkPoolWorker-5415:
Process Keras_worker_ForkPoolWorker-5404:
Process Keras_worker_ForkPoolWorker-5421:
Process Keras_worker_ForkPoolWorker-5408:
Process Keras_worker_ForkPoolWorker-5407:
Process Keras_worker_ForkPoolWorker-5402:
Process Keras_worker_ForkPoolWorker-5424:
Process Keras_worker_ForkPoolWorker-5409:
Process Keras_worker_ForkPoolWorker-5406:
Process Keras_worker_ForkPoolWorker-5418:
Process Keras_worker_ForkPoolWorker-5416:
Process Keras_worker_ForkPoolWorker-5401:
Process Keras_worker_ForkPoolWorker-5420:
Process Keras_worker_ForkPoolWorker-5423:
Process Keras_worker_ForkPoolWorker-5412:
Process Keras_worker_ForkPoolWorker-5413:
Process Keras_worker_ForkPoolWorker-5425:
Process Keras_worker_ForkPoolWorker-5411:
Process Keras_worker_ForkPoolWorker-5410:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most rece

KeyboardInterrupt: 

In [8]:
#54 Epoches
seq=load_model("STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-54-15.19-8.32.hdf5")
seq.evaluate_generator(valid_gen,n_frames_valid/batch_size/2, max_queue_size = 80,workers = 10,verbose=2)

Instructions for updating:
Please use Model.evaluate, which supports generators.
45/44 - 43s - loss: 15.0377


15.037659645080566

In [9]:
#53 Epoches
seq=load_model("STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-53-13.79-8.44.hdf5")
seq.evaluate_generator(valid_gen,n_frames_valid/batch_size/2, max_queue_size = 80,workers = 10,verbose=2)

45/44 - 46s - loss: 13.6259


13.625862121582031

In [10]:
#21 Epoches
seq=load_model("STACKED_CNN_2FPS_Block_approach_45bs-HE_norm-op_flow_dense-RGB-KITTI-21-16.70-16.22.hdf5")
seq.evaluate_generator(valid_gen,n_frames_valid/batch_size/2, max_queue_size = 80,workers = 10,verbose=2)

45/44 - 46s - loss: 16.5304


16.530378341674805